In [ ]:
#following load /user/ICML_gama_2d.xlsx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import re
import math
a=pd.read_excel('/user_control/paper_results/ICML_gama_2d.xlsx',sheet_name='Sheet1')
# an example element of a in colume 1 is 0.1, and an example element of a in colume 2,3,4 is 0.1(0.2) mean(std)
#followng use the first colume as the x axis, and the second colume as the y axis,plot x,y
#transform a into np.array
a=np.array(a)
print(a)
# Function to transform each row
def transform_row(row):
    transformed_row = [float(row[0])]  # Adding the first element as is (assuming it's always a float)
    
    # Extracting numerical values from the string elements
    for element in row[1:]:
        numerical_value = float(element.split()[0])  # Extracting the numerical part before the space
        transformed_row.append(numerical_value)
    
    return transformed_row
transformed_a = np.apply_along_axis(transform_row, axis=1, arr=a)

print(transformed_a)
# then plot the transformed_a
x=transformed_a[:,0]
y=transformed_a[:,1]
z=transformed_a[:,2]
w=transformed_a[:,3]

# plot x,y
plt.plot(x,y,'r',label='mean')
plt.legend()
#save fig as pdf
plt.savefig('mean.pdf')

In [ ]:
# transformed_a[:,0]=np.log10(transformed_a[:,0])
print(transformed_a[:,1])
fontsize=18
plt.figure()

plt.plot(transformed_a[:,0],transformed_a[:,1], label=r'$\bar{v}$', marker='o',color='blue')



plt.legend()
# plt.title(r'design obj of different $\lambda$')
plt.xlabel(r'$\gamma$',fontsize=fontsize)
plt.ylabel(r'$\bar{v}$',fontsize=fontsize)
plt.tick_params(labelsize=12)
plt.ylim(0,2.2)
plt.savefig('/user/pde_gen_control/plot_results/v_bar_of_different_gama.pdf')

In [ ]:
#follow plot a[:,0],a[:,2](which is the $R(\theta)$)
fontsize=18
plt.figure()
plt.plot(transformed_a[:,0],transformed_a[:,2], label=r'$R(\theta)$', marker='o',color='blue')
plt.legend()
# plt.title(r'design obj of different $\lambda$')
plt.xlabel(r'$\gamma$',fontsize=fontsize)
plt.ylabel(r'$R(\theta)$',fontsize=fontsize)
plt.tick_params(labelsize=12)
# plt.ylim(0,2.2)
plt.savefig('/user/plot_results/R_theta_different_gama.pdf')


In [ ]:
#follow plot a[:,0],a[:,3](which is the obj)
fontsize=18
plt.figure()
plt.plot(transformed_a[:,0],transformed_a[:,3], label=r'obj', marker='o',color='blue')
plt.legend()
plt.xlabel(r'$\gamma$',fontsize=fontsize)
plt.ylabel(r'obj',fontsize=fontsize)
plt.tick_params(labelsize=12)
# plt.ylim(0,2.2)
plt.savefig('/user/plot_results/obj_different_gama.pdf')

In [ ]:
# then plot different scale level into one figure
transformed_a[1,-1]=-74.11
print(transformed_a)
plt.figure()
plt.plot(transformed_a[:,0],transformed_a[:,1], label=r'$\bar{v}$', marker='o',color='blue')
plt.plot(transformed_a[:,0],transformed_a[:,2]*1000, label=r'$1000*R(\theta)$', marker='*',color='red')
#following plot transformed_a[:,3] into the same figure with different y axis
ax1 = plt.gca()
ax2 = ax1.twinx()
ax2.plot(transformed_a[:,0],transformed_a[:,3], label=r'obj', marker='s',color='green') 
# plt.plot(transformed_a[:,0],transformed_a[:,3], label=r'obj', marker='o',color='green')
ax1.legend(loc='upper left')
ax2.legend()
plt.legend()
# plt.title(r'design obj of different $\lambda$')
plt.xlim(0.5,1.6)
plt.xlabel(r'$\xi$',fontsize=fontsize)
ax1.set_ylim(0,500)
ax2.set_ylim(-160,50)
ax1.set_ylabel(r'$\bar{v}$, $R(\theta)$',fontsize=fontsize)    
ax2.set_ylabel(r'obj',fontsize=fontsize)
ax1.set_xlabel(r'$\gamma_0$',fontsize=fontsize)
# ax1.get_ylim(0,500)
plt.tick_params(labelsize=12)
# plt.ylim(0,2.2)
plt.savefig('/user_control/paper_results/v_bar_R_theta_obj_different_gama.pdf',bbox_inches='tight')

In [ ]:
#following load /user/project/pde_gen_control/paper_results/compare_theta_curves/compart_thetas_different_methods.pkl
import pickle
with open('/user/project/pde_gen_control/paper_results/compare_theta_curves/compart_thetas_different_methods.pkl', 'rb') as f:
    compart_thetas_different_methods = pickle.load(f)

print(compart_thetas_different_methods[3])

In [ ]:
import pickle
import matplotlib.pyplot as plt

# Correct file path to the uploaded .pkl file with the correct extension
file_path = '/user_control/paper_results/compare_theta_curves/compart_thetas_different_methods.pkl'
# Load the contents from the file
# Replace 'file_path' with the path to your .pkl file.
with open(file_path, 'rb') as file:
    results = pickle.load(file)

fig, axs = plt.subplots(1, 3, figsize=(20, 4), sharey=True)
methods = ["1ddpm", "2ddpm", "SAC_online", "SAC_offline", "SL", "MPC"]
x_ticks = [0, 5, 10, 15, 20]
y_ticks = [0, 20, 40, 60, 80]
total_handle = []
total_labels = []
sample_id_list=[4,5,1]
for idx, sample_id in enumerate(sorted(results.keys())):
    each_handle = []
    each_label = []
    for i, method in enumerate(methods):
        if idx == 3:
            break
        sample_id=sample_id_list[idx]
        theta_values = results[sample_id][method]['theta']
        obj_values = results[sample_id][method]['obj']
        # Rename method for display
        display_name = {"1ddpm": "DiffConPDE-lite", "2ddpm": "DiffConPDE",
                        "SAC_online": "SAC (pseudo-online)", "SAC_offline": "SAC (offline)"}.get(method, method)
        
        #scatter the first theta and last timesteps theta use the same color as the line
        if method=="1ddpm" or method=="2ddpm":
            line, = axs[idx].plot(theta_values, label=display_name)
            axs[idx].scatter(0, theta_values[0], color=line.get_color())
            axs[idx].scatter(19, theta_values[-1], color=line.get_color())
        else:
            line, = axs[idx].plot(theta_values, label=display_name,linestyle='--')
        # axs[idx].set_title(f"Sample ID: {sample_id}")
        
        if idx == 0:
            total_handle.append(line)
            total_labels.append(display_name)
        each_handle.append(line)
        each_label.append(r"$J$: "+f"{obj_values:.2f}")
    if idx!=3:
        axs[idx].set_xlim(-1, 21)
        axs[idx].set_xticks(x_ticks)
        axs[idx].set_ylim(0, 80)
        axs[idx].set_yticks(y_ticks)
        axs[idx].tick_params(axis='both', which='both', labelsize=10, labelleft=True, labelbottom=True)
        legend1 = axs[idx].legend(handles=each_handle[:2], labels=each_label[:2], loc='upper left', fontsize='large')
        axs[idx].add_artist(legend1)
        legend2= axs[idx].legend(handles=each_handle[2:4], labels=each_label[2:4], loc='upper center', fontsize='large')
        axs[idx].add_artist(legend2)
        axs[idx].legend(handles=each_handle[4:], labels=each_label[4:], loc='upper right', fontsize='large')
        axs[idx].set_xlabel(r"$t$", fontsize='xx-large')
        axs[idx].set_ylabel(r"$w$", fontsize='xx-large')

# Adjust the layout to make space for the overall legend
fig.subplots_adjust(bottom=0.21)
fig.legend(total_handle, total_labels, loc='lower center', ncol=6, bbox_to_anchor=(0.5, -0.05), fontsize='x-large')
fig.savefig('/user_control/paper_results/2d_test_sample_theta.pdf', bbox_inches='tight')
plt.show()


In [ ]:
from PIL import Image

def crop_center_numpy(img, cropx, cropy):
    y, x, _ = img.shape
    print('ssss',x)
    startx = x // 2 - (cropx // 2)
    starty = y // 2 - (cropy // 2)    
    return img[starty:starty+cropy, startx:startx+cropx]

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle

# Sample and time IDs
sample_id = 5
time_ids = [0, 50, 100, 150, 200]

# Set up the subplots
fig, axs = plt.subplots(1, len(time_ids), figsize=(15, 5))

# Load and display each image with a black border
for ax, time_id in zip(axs, time_ids):
    img_path = f'/usert/diffusion_operator/results/density_sample/density_WDNO{3}_{time_id}.png'  # Replace with actual path
    img = mpimg.imread(img_path)
    # cropped_img = crop_center_numpy(img, cropx=100, cropy=100)
    cropped_img=img[200:-200,150:-150]
    ax.imshow(cropped_img)
    ax.axis('off')  # Hide the axes
    if time_id==39:
        time_id=40
    ax.set_title(f"t = {time_id}", fontsize=21)
    # Add a black border around the image
    rect = Rectangle((0, 0), 1, 1, fill=False, edgecolor='black', linewidth=5, transform=ax.transAxes)
    ax.add_patch(rect)

plt.tight_layout()
plt.show()

# Save the figure
# fig.savefig('/usert/diffusion_operator/results/phiflow_draw/2d_phiflow.pdf',bbox_inches="tight")  # Replace with the actual path where you want to save the figure


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle

# Sample and time IDs
sample_id = 5
time_ids = [0, 56, 112, 168, 224]

# Set up the subplots
fig, axs = plt.subplots(1, len(time_ids), figsize=(15, 5))

# Load and display each image with a black border
for ax, time_id in zip(axs, time_ids):
    img_path = f'/usert/diffusion_operator/results/density_sample/density_WDNO{3}_{time_id}.png'  # Replace with actual path
    img = mpimg.imread(img_path)
    cropped_img = img[200:, 300:-300]
    # cropped_img = img
    ax.imshow(cropped_img)
    ax.axis('off')  # Hide the axes
    if time_id == 39:
        time_id = 40
    ax.set_title(f"t = {int(time_id/8)}", fontsize=21)

plt.subplots_adjust(wspace=-0.2, hspace=0)  # Adjust the space between subplots
plt.tight_layout()
plt.show()

# Save the figure
fig.savefig('/usert/diffusion_operator/results/phiflow_draw/2d_phiflow.pdf', bbox_inches="tight")  # Replace with the actual path where you want to save the figure


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Sample and time IDs
sample_ids = [3, 4, 5, 6, 14, 16,19]
time_ids = [0, 56, 112, 168, 224]

# Set up the subplots
fig, axs = plt.subplots(len(sample_ids), len(time_ids), figsize=(15, 21))

# Load and display each image with a black border
for i, sample_id in enumerate(sample_ids):
    for j, time_id in enumerate(time_ids):
        img_path = f'/usert/diffusion_operator/results/dens_sample_20/density_WDNO{sample_id}_{time_id}.png'  # Replace with actual path
        img = mpimg.imread(img_path)
        cropped_img = img[200:, 300:-300]
        axs[i, j].imshow(cropped_img)
        axs[i, j].axis('off')  # Hide the axes
        if i == 0:
            axs[i, j].set_title(f"t = {int(time_id/8)}", fontsize=21)

plt.subplots_adjust(wspace=-0.2, hspace=0)  # Adjust the space between subplots
plt.tight_layout()
plt.show()

# Save the figure
fig.savefig('/usert/diffusion_operator/results/phiflow_draw/2d_phiflow_multiple.pdf', bbox_inches="tight")  # Replace with the actual path where you want to save the figure


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
# Sample and time IDs
sample_ids = [0, 1, 2, 3, 4]
time_ids = [20, 25, 30, 35, 39]

# 设置子图大小
fig, axs = plt.subplots(len(sample_ids), len(time_ids), figsize=(20, 22))
fig.subplots_adjust(hspace=0.1)
for i, sample_id in enumerate(sample_ids):
    for j, time_id in enumerate(time_ids):
        # 计算子图的索引
        ax = axs[i, j]
        
        # 加载并展示每个图像
        img_path = f'/user_control/paper_results/2d_fluid_visualization/sim_{sample_id}_t_{time_id}.png'  # 根据实际路径替换
        img = mpimg.imread(img_path)
        ax.imshow(img)
        ax.axis('off')  # 隐藏坐标轴
        if i==0:
            if time_id==39:
                time_id=40
            ax.set_title(f"t = {time_id-20}", fontsize=22)
        # 为图像添加黑色边框
        rect = Rectangle((0, 0), 1, 1, fill=False, edgecolor='black', linewidth=4, transform=ax.transAxes)
        ax.add_patch(rect)

# 调整布局
plt.tight_layout()
plt.show()

# 保存图像
fig.savefig('/user_control/paper_results/multi_sflow_field.pdf')


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle

# Sample and time IDs
sample_id = 5
time_ids = [0, 1,2,3]

# Set up the subplots
fig, axs = plt.subplots(1, len(time_ids), figsize=(17, 4))
title_list=['No upsampling','Single upsampling','Double upsampling','Triple upsampling']
# Load and display each image with a black border
for ax, time_id in zip(axs, time_ids):
    img_path = f'/usert/diffusion_operator/results/vis_1d_super/z_plot_wave_True_{time_id}_4.jpg'  # Replace with actual path
    img = mpimg.imread(img_path)
    cropped_img = img[1600:3600, 2000:4000]
    # cropped_img = img
    ax.imshow(cropped_img)
    ax.axis('off')  # Hide the axes
    if time_id == 39:
        time_id = 40
    ax.set_title(title_list[time_id], fontsize=21)
    rect = Rectangle((0, 0), 1, 1, fill=False, edgecolor='black', linewidth=3, transform=ax.transAxes)
    ax.add_patch(rect)

plt.subplots_adjust(wspace=-0.2, hspace=0.1)  # Adjust the space between subplots
plt.tight_layout()
plt.show()

# Save the figure
fig.savefig('/usert/diffusion_operator/results/phiflow_draw/2d_phiflow_super_resolution.pdf', bbox_inches="tight")  # Replace with the actual path where you want to save the figure


In [ ]:
print(img.shape)